In [1]:
import tensorflow as tf
from skimage.io import imread
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

/Users/thomasorkild/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
vgg16 = tf.keras.applications.VGG16(include_top=False,weights='imagenet')
vgg16_graph = tf.keras.backend.get_session().graph

In [3]:
w_1_1 = vgg16_graph.get_tensor_by_name('block1_conv1/kernel:0')
b_1_1 = vgg16_graph.get_tensor_by_name('block1_conv1/bias:0')
w_1_2 = vgg16_graph.get_tensor_by_name('block1_conv2/kernel:0')
b_1_2 = vgg16_graph.get_tensor_by_name('block1_conv2/bias:0')

w_2_1 = vgg16_graph.get_tensor_by_name('block2_conv1/kernel:0')
b_2_1 = vgg16_graph.get_tensor_by_name('block2_conv1/bias:0')
w_2_2 = vgg16_graph.get_tensor_by_name('block2_conv2/kernel:0')
b_2_2 = vgg16_graph.get_tensor_by_name('block2_conv2/bias:0')

w_3_1 = vgg16_graph.get_tensor_by_name('block3_conv1/kernel:0')
b_3_1 = vgg16_graph.get_tensor_by_name('block3_conv1/bias:0')
w_3_2 = vgg16_graph.get_tensor_by_name('block3_conv2/kernel:0')
b_3_2 = vgg16_graph.get_tensor_by_name('block3_conv2/bias:0')
w_3_3 = vgg16_graph.get_tensor_by_name('block3_conv3/kernel:0')
b_3_3 = vgg16_graph.get_tensor_by_name('block3_conv3/bias:0')

w_4_1 = vgg16_graph.get_tensor_by_name('block4_conv1/kernel:0')
b_4_1 = vgg16_graph.get_tensor_by_name('block4_conv1/bias:0')
w_4_2 = vgg16_graph.get_tensor_by_name('block4_conv2/kernel:0')
b_4_2 = vgg16_graph.get_tensor_by_name('block4_conv2/bias:0')
w_4_3 = vgg16_graph.get_tensor_by_name('block4_conv3/kernel:0')
b_4_3 = vgg16_graph.get_tensor_by_name('block4_conv3/bias:0')

w_5_1 = vgg16_graph.get_tensor_by_name('block5_conv1/kernel:0')
b_5_1 = vgg16_graph.get_tensor_by_name('block5_conv1/bias:0')
w_5_2 = vgg16_graph.get_tensor_by_name('block5_conv2/kernel:0')
b_5_2 = vgg16_graph.get_tensor_by_name('block5_conv2/bias:0')
w_5_3 = vgg16_graph.get_tensor_by_name('block5_conv3/kernel:0')
b_5_3 = vgg16_graph.get_tensor_by_name('block5_conv3/bias:0')

In [4]:
w_4_1

<tf.Tensor 'block4_conv1/kernel:0' shape=(3, 3, 256, 512) dtype=float32_ref>

In [11]:
images=tf.placeholder(dtype=tf.float32,shape=(None, None,None, 3))
strides = (1,1,1,1)
padding = 'SAME'
max_pooling_ksize = (1,2,2,1)

In [12]:
conv_1_1 = tf.nn.bias_add(tf.nn.conv2d(input = images,filter=w_1_1, strides=strides, padding=padding), b_1_1)
conv_1_2 = tf.nn.bias_add(tf.nn.conv2d(input = conv_1_1,filter=w_1_2, strides=strides, padding=padding), b_1_2)
max_pooling_1 = tf.nn.max_pool(conv_1_2, ksize=max_pooling_ksize,strides=strides, padding='VALID')

conv_2_1 = tf.nn.bias_add(tf.nn.conv2d(input = max_pooling_1,filter=w_2_1, strides=strides, padding=padding), b_2_1)
conv_2_2 = tf.nn.bias_add(tf.nn.conv2d(input = conv_2_1,filter=w_2_2, strides=strides, padding=padding), b_2_2)
max_pooling_2 = tf.nn.max_pool(conv_2_2, ksize=max_pooling_ksize,strides=strides, padding='VALID')

conv_3_1 = tf.nn.bias_add(tf.nn.conv2d(input = max_pooling_2,filter=w_3_1, strides=strides, padding=padding), b_3_1)
conv_3_2 = tf.nn.bias_add(tf.nn.conv2d(input = conv_3_1,filter=w_3_2, strides=strides, padding=padding), b_3_2)
conv_3_3 = tf.nn.bias_add(tf.nn.conv2d(input = conv_3_2,filter=w_3_3, strides=strides, padding=padding), b_3_3)
max_pooling_3 = tf.nn.max_pool(conv_3_3, ksize=max_pooling_ksize,strides=strides, padding='VALID')

conv_4_1 = tf.nn.bias_add(tf.nn.conv2d(input = max_pooling_3,filter=w_4_1, strides=strides, padding=padding), b_4_1)
conv_4_2 = tf.nn.bias_add(tf.nn.conv2d(input = conv_4_1,filter=w_4_2, strides=strides, padding=padding), b_4_2)
conv_4_3 = tf.nn.bias_add(tf.nn.conv2d(input = conv_4_2,filter=w_4_3, strides=strides, padding=padding), b_4_3)
max_pooling_4 = tf.nn.max_pool(conv_4_3, ksize=max_pooling_ksize,strides=strides, padding='VALID')

conv_5_1 = tf.nn.bias_add(tf.nn.conv2d(input = max_pooling_4,filter=w_5_1, strides=strides, padding=padding), b_5_1)
conv_5_2 = tf.nn.bias_add(tf.nn.conv2d(input = conv_5_1,filter=w_5_2, strides=strides, padding=padding), b_5_2)
conv_5_3 = tf.nn.bias_add(tf.nn.conv2d(input = conv_5_2,filter=w_5_3, strides=strides, padding=padding), b_5_3)
max_pooling_5 = tf.nn.max_pool(conv_5_3, ksize=max_pooling_ksize,strides=strides, padding='VALID')

In [13]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [ ]:
image = imread('../Data/Raw_images/0001TP_006690.png')
image = np.reshape(image,(1,np.shape(image)[0],np.shape(image)[1],3))
sess.run(max_pooling_5,feed_dict={images: image})